# Function Calling / Tool Calling

In [1]:
from openai import AzureOpenAI
client = AzureOpenAI(api_version="2024-12-01-preview")

In [3]:
message = [{'role':'user','content':'what is Quantum Computing.'}]

response = client.chat.completions.create(model='telcogpt',
                                          messages =message,temperature=0.1)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-BWe5btzp1EPSPXPmMPPY9x7bSHnIz",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Quantum computing is a type of computation that takes advantage of the principles of quantum mechanics to process information in fundamentally different ways than classical computers. Here are some key concepts to understand:\n\n1. **Quantum Bits (Qubits)**: Unlike classical bits, which can be either 0 or 1, qubits can exist in a state of superposition, meaning they can represent both 0 and 1 simultaneously. This property allows quantum computers to process a vast amount of information at once.\n\n2. **Superposition**: This principle allows qubits to be in multiple states at the same time. When a quantum computer performs calculations, it can explore many possible solutions simultaneously, potentially leading to faster problem-solving for certain tasks.\n\n3. **Entanglement**: Qubits can become entangled

In [12]:
import requests, json, time

def get_current_weather(city:str)->dict:
    """ this funciton can be used to get current weather information"""
    api_key="6a8b0ac166a37e2b7a38e64416b3c3fe"
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = json.loads(response.content.decode())
    output = {"city":city,"weather":response['weather'][0]['description'],
              "temperature":response['main']['temp'], "unit":"kelvin"
              }
    return output

In [13]:
get_current_weather("delhi")

{'city': 'delhi',
 'weather': 'clear sky',
 'temperature': 312.12,
 'unit': 'kelvin'}

In [15]:
# 3 components : name of function, description, parameters (desc of params)
tools = [{
    "type":"function",
    "function":{
        "name":"get_current_weather_info",
        "description":"This function can be used to get current weather information for any given city.",
        "parameters":{
            "type":"object",
            "properties":{"city":{"type":"string","description":"name of location/city e.g. delhi, mumbai"}}
        },
        "required":['city',]

    }
},
]

In [25]:
tool_map = {"get_current_weather_info":get_current_weather}

def generate_response(prompt):

    messages=[{"role":"system","content":"you are a helpful assistant you provide short & concise answers"},
              {"role":"user",'content':prompt}]
    
    first_response = client.chat.completions.create(model="telcogpt",
                                                    messages=messages,temperature=0.2,
                                                    tools=tools,tool_choice='auto')
    if first_response.choices[0].message.tool_calls:
        # do something
        tool_calls = first_response.choices[0].message.tool_calls
        print(tool_calls)
        messages.append(first_response.choices[0].message)

        for tool in tool_calls:
            tool_name = tool.function.name
            tool_args = json.loads(tool.function.arguments)
            func = tool_map[tool_name]
            tool_res = func(**tool_args)
            messages.append({"tool_call_id":tool.id,"role":"tool","name":tool_name,
                             "content":json.dumps(tool_res)})
        
        second_response = client.chat.completions.create(messages=messages,model='telcogpt',
                                                         temperature=0.1)
        return second_response.choices[0].message.content

    else:
        return first_response.choices[0].message.content

In [23]:
resp = generate_response("what is quantum computing")
print(resp)

Quantum computing is a type of computing that uses quantum bits, or qubits, which can represent and store information in ways that classical bits cannot. Unlike classical bits that are either 0 or 1, qubits can exist in multiple states simultaneously due to the principles of superposition and entanglement. This allows quantum computers to perform complex calculations much faster than classical computers for certain problems, such as factoring large numbers and simulating quantum systems.


In [26]:
resp = generate_response("what is the current weather in bangalore")
print(resp)

[ChatCompletionMessageToolCall(id='call_PI6aGjCqyeC2M3gqG28IEski', function=Function(arguments='{"city":"bangalore"}', name='get_current_weather_info'), type='function')]
The current weather in Bangalore is clear sky with a temperature of 305 K.


In [ ]:
resp = generate_response("what is the current weather in bangalore and london")
print(resp)

[ChatCompletionMessageToolCall(id='call_DbSllSk5Q2Im4gLv11IjhxoW', function=Function(arguments='{"city": "bangalore"}', name='get_current_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_lVeC45mH7lf2APFib2XszDBp', function=Function(arguments='{"city": "london"}', name='get_current_weather_info'), type='function')]
